# Data Preprocessing

Min-max scaling of our features

In [1]:
import pandas as pd
import numpy as np  
import os

In [2]:
# For reproducibility of results
np.random.seed(0)

# Load the dataset if selected_features.csv does not exist
if os.path.exists("selected_features.csv"):
    selected_features = pd.read_csv("selected_features.csv")
    print(selected_features.head())
else:
    file_path = "../DataProcessing/rtfDataSet.csv"
    original_data = pd.read_csv(file_path)
    # Get only our features of interest and save them to a local file "selecte_features.csv"
    # 2 7 8 11 14 15 17 18 21 22 23 25 30 31 33 36 40 43 44 50 53 54 56 57 58 59 60 64 66 68
    selected_features = original_data.iloc[:, [2, 7, 8, 11, 14, 15, 17, 18, 21, 22, 23, 25, 30, 31, 33, 36, 40, 43, 44, 50, 53, 54, 56, 57, 58, 59, 60, 64, 66, 68]]    
    # Save the selected features to a local file
    selected_features.to_csv("selected_features.csv", index=False)
    print(original_data.head())
    print(selected_features.head())

       2      7      8     11     14     15     17     18     21     22  ...  \
0 -0.684 -0.113  0.401 -0.251 -0.178  0.321  0.016 -0.003 -0.275 -0.162  ...   
1 -0.529 -0.066 -0.168 -0.205  0.020  0.031 -0.165 -0.026 -0.130  0.176  ...   
2 -0.240  0.031 -0.077 -0.036 -0.005  0.170  0.212  0.138 -0.300  0.689  ...   
3 -0.062 -0.143  0.068 -0.295 -0.188 -0.317 -0.113  0.018 -0.272 -0.531  ...   
4 -0.203 -0.168  0.008  0.166 -0.048 -0.291 -0.091  0.062  0.026 -0.036  ...   

      53     54     56     57     58     59     60     64     66     68  
0 -0.334 -0.263 -0.075 -0.327 -0.198 -0.257 -0.205 -0.223 -0.136 -0.132  
1  0.216 -0.064  0.004 -0.305 -0.018 -0.230 -0.150 -0.151 -0.031 -0.097  
2 -0.027 -0.149  0.102 -0.189 -0.112 -0.121  0.121 -0.172  0.033 -0.321  
3  0.389 -0.245 -0.153 -0.405 -0.102 -0.120 -0.141 -0.292 -0.078 -0.151  
4  0.032  0.043 -0.099 -0.188  0.042 -0.124 -0.097 -0.165 -0.112 -0.138  

[5 rows x 30 columns]


## Min-Max

In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_normalized = scaler.fit_transform(selected_features)

## Stratified 10-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

# TODO: Initialize your features as X and labels as y
# X = ...
# y = ...

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=10)

# TODO: Implement the loop for Stratified 10-Fold CV
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # TODO: Insert your classifier training and testing code here


## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Initialize classifier
nb = GaussianNB()

# TODO: Fit the model with training data
# nb.fit(X_train, y_train)

# TODO: Make predictions on test data
# y_pred = nb.predict(X_test)


## k-Nearest Neighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize classifier with k value and metric type
# knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')

# TODO: Fit the model with training data
# knn.fit(X_train, y_train)

# TODO: Make predictions on test data
# y_pred = knn.predict(X_test)
